<a href="https://colab.research.google.com/github/manasi-ganti/long-doc-summarization-gpt/blob/main/lds_sota.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python3 -m pip install datasets
!python3 -m pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python3 -m pip install sentencepiece
!python3 -m pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_dataset_builder, load_dataset
from datasets import get_dataset_config_names
from datasets import get_dataset_split_names

from tqdm.auto import tqdm  # for our loading bar
import pandas as pd
import nltk
nltk.download('punkt')

from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
    

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [ ]:
#LOAD DATASET + MODEL
dataset_name = "ccdv/arxiv-summarization" # "ccdv/arxiv-summarization"
configs = get_dataset_config_names(dataset_name)
print(configs)

ds_builder = load_dataset_builder(dataset_name, "document")
print(ds_builder.info.description)
print(ds_builder.info.features)

dataset = load_dataset(dataset_name)
print(dataset)
print(dataset['train'][1])

small_train_dataset = dataset["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = dataset["validation"].shuffle(seed=42).select(range(100))
small_test_dataset = dataset["test"].shuffle(seed=42).select(range(100))


['section', 'document']

 Arxiv dataset for summarization.
 From paper: A Discourse-Aware Attention Model for Abstractive Summarization of Long Documents" by A. Cohan et al.
 See: https://aclanthology.org/N18-2097.pdf 
 See: https://github.com/armancohan/long-summarization

{'article': Value(dtype='string', id=None), 'abstract': Value(dtype='string', id=None)}


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract'],
        num_rows: 203037
    })
    validation: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6436
    })
    test: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6440
    })
})
{'article': 'the leptonic decays of a charged pseudoscalar meson @xmath7 are processes of the type @xmath8 , where @xmath9 , @xmath10 , or @xmath11 . because no strong interactions are present in the leptonic final state @xmath12 , such decays provide a clean way to probe the complex , strong interactions that bind the quark and antiquark within the initial - state meson . in these decays , strong interaction effects can be parametrized by a single quantity , @xmath13 , the pseudoscalar meson decay constant . \n the leptonic decay rate can be measured by experiment , and the decay constant can be determined by the equation ( ignoring radiative corrections ) @xmath14 where @xmath15 is th

In [ ]:
model_name = "google/pegasus-pubmed" # "google/pegasus-arxiv"
device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
tokenizer = PegasusTokenizer.from_pretrained(model_name)       
model = PegasusForConditionalGeneration.from_pretrained(model_name)#.to(device)

In [ ]:
torch.cuda.empty_cache()


In [ ]:
batch = tokenizer(small_eval_dataset["article"][0:10], truncation=True, padding="longest", return_tensors="pt").to(device)
summaries = model.generate(**batch)
print(summaries)
summaries_decoded = tokenizer.batch_decode(summaries, skip_special_tokens=True)
print(len(summaries_decoded))

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1232: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 256 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


tensor([[    0,   122,   109,  ...,     0,     0,     0],
        [    0,   109,   613,  ...,  3627,   135,     1],
        [    0,   109,  9498,  ...,     0,     0,     0],
        ...,
        [    0,   145,  1037,  ...,   117,   146,     1],
        [    0, 21312, 10135,  ...,   757, 46819,     1],
        [    0,   115,   114,  ..., 10287,   114,     1]])
10


In [ ]:
from google.colab import drive
drive.mount('drive')


def get_num_words(sentence):
  print(len(sentence.split()))
  return len(sentence.split())

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
data = [summaries_decoded, small_eval_dataset["article"][0:10], small_eval_dataset["abstract"][0:10]]
output = pd.DataFrame(data)
output.to_csv('out_arxiv_pegapubmed.csv', index = False)  
!cp out_arxiv_pegapubmed.csv "drive/My Drive/model output"


In [ ]:
from datasets import load_metric

metric = load_metric("rouge")
metric.compute(predictions=summaries_decoded, references=small_eval_dataset["abstract"][0:10])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  This is separate from the ipykernel package so we can avoid doing imports until


{'rouge1': AggregateScore(low=Score(precision=0.3297318362722128, recall=0.37571958692094853, fmeasure=0.34938808189621456), mid=Score(precision=0.41103223688442536, recall=0.4606072496724332, fmeasure=0.41290229119500926), high=Score(precision=0.504641645417791, recall=0.540496374250508, fmeasure=0.4982351256447917)),
 'rouge2': AggregateScore(low=Score(precision=0.10890657372020254, recall=0.12659358538750734, fmeasure=0.11288098380240152), mid=Score(precision=0.1703658650548369, recall=0.1896564356999355, fmeasure=0.17258321220086842), high=Score(precision=0.2565072796239066, recall=0.26755406390370445, fmeasure=0.2557864182984332)),
 'rougeL': AggregateScore(low=Score(precision=0.17715492159964683, recall=0.20074026109698137, fmeasure=0.1852854439133198), mid=Score(precision=0.22524844648420078, recall=0.2535185538385293, fmeasure=0.22770130403990696), high=Score(precision=0.2991746860887209, recall=0.3130637759100944, fmeasure=0.2895474793561775)),
 'rougeLsum': AggregateScore(low

In [ ]:
print(hi)
max_input_length = 3000
max_target_length = 200

def preprocess_function(examples):
    #inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(examples["article"], max_length=max_input_length, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["abstract"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

#tokenized_data = dataset.map(preprocess_function, batched=True)
#tokenized_data_val2 = dataset['validation'].map(preprocess_function, batched=True)
#tokenized_dataset_train = small_train_dataset.map(preprocess_function, batched=True) #dataset['train']
tokenized_dataset_val = small_eval_dataset.map(preprocess_function, batched=True) #dataset['validation']
#tokenized_dataset_test = small_test_dataset.map(preprocess_function, batched=True) #dataset['test']

In [ ]:
print(tokenized_dataset_val)

In [ ]:
print(tokenized_dataset_val["labels"])
#tokenized_dataset_val.rename_column("input_ids", '2')
import numpy as np
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)


#tokenized_dataset_val = {int(k):v for k,v in tokenized_dataset_val.items()}
summaries = model.generate(tokenized_dataset_val["labels"])#['input_ids']) #torch.Tensor(tokenized_dataset_val[0:100][0]))
print(summaries)

In [ ]:
print(summaries.dtype)

In [ ]:
from tqdm.auto import tqdm  # for our loading bar

text_data = []
batch_count = 0
batches = []
for sample in tqdm(dataset['validation']):
    # remove newline characters from each sample as we need to use exclusively as seperators
    sample = sample['article'].replace('\n', '')
    text_data.append(sample)


In [ ]:
text = "Replace me by any text you'd like."
#inputs = tokenizer(text_data[0:10], return_tensors='pt', padding=True, truncation=True)
prediction = model.generate(tokenized_dataset_val[0:100])#model.generate(tokenized_datasets["train"][0:])
prediction = tokenizer.batch_decode(prediction)
print(prediction)



In [ ]:
x

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

batch_size = 16
model_name = "google/pegasus-pubmed".split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)




import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}





trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
from transformers import TrainingArguments, Trainer

from datasets import load_metric

rouge = load_metric("rouge")


small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))


training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    #compute_metrics=compute_metrics,
)

trainer.train()





from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

#training_args = Seq2SeqTrainingArguments(
#    output_dir="./results",
#    evaluation_strategy="epoch",
#    learning_rate=2e-5,
#    per_device_train_batch_size=1000,
#    per_device_eval_batch_size=500,
#    weight_decay=0.01,
#    save_total_limit=3,
#    num_train_epochs=1,
#)


from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

print(tokenized_datasets["train"][0])

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)


trainer.train()

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

from transformers import TrainingArguments, Trainer

from datasets import load_metric

rouge = load_metric("rouge")

model_name = "google/pegasus-pubmed"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

#TOKENIZE
def preprocess_function(examples):

    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["abstract"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)





In [ ]:
print(tokenized_dataset["train"][0])


In [ ]:
#pegasus
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "google/pegasus-xsum"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

def tokenize_function(samples):
    return tokenizer(samples['article'].replace('\n', ''), padding="max_length", truncation=True)
#def preprocess_function(examples):
#    inputs = [prefix + doc for doc in examples["document"]]
#    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
#    with tokenizer.as_target_tokenizer():
#        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

 #   model_inputs["labels"] = labels["input_ids"]
 #   return model_inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)
#tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)






small_train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(1000))

from transformers import TrainingArguments, Trainer

from datasets import load_metric

rouge = load_metric("rouge")
import numpy as np
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return rouge.compute(predictions=predictions, references=labels)


training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset#,
    compute_metrics=compute_metrics,
)

trainer.train()


In [ ]:
#batch = tokenizer(text_data, truncation=True, padding="longest", return_tensors="pt").to(device)


In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="google/pegasus-xsum", tokenizer="google/pegasus-xsum", framework="pt")
summarizer(text_data)
